In [1]:
import logging
import lmdb
from gensim.models import Word2Vec
from doc_pb2 import Document, Dictionary

logging.basicConfig(format='%(asctime)s : %(levelname)s : %(message)s', level=logging.INFO)

C:\Anaconda3\lib\site-packages\gensim\utils.py:865: UserWarning: detected Windows; aliasing chunkize to chunkize_serial
  warnings.warn("detected Windows; aliasing chunkize to chunkize_serial")
Using TensorFlow backend.


In [83]:
%%time
dictionary = Dictionary()
with open ('dumps/full_dict.dump', 'rb') as f:
    dictionary.ParseFromString(f.read())

Wall time: 3min 56s


In [4]:
%%time
model = Word2Vec.load('dumps/news.model ')

2017-08-10 13:26:15,473 : INFO : loading Word2Vec object from dumps/news.model 
2017-08-10 13:26:16,278 : INFO : loading wv recursively from dumps/news.model .wv.* with mmap=None
2017-08-10 13:26:16,279 : INFO : loading syn0 from dumps/news.model .wv.syn0.npy with mmap=None
2017-08-10 13:26:17,930 : INFO : setting ignored attribute syn0norm to None
2017-08-10 13:26:17,931 : INFO : loading syn1neg from dumps/news.model .syn1neg.npy with mmap=None
2017-08-10 13:26:19,580 : INFO : setting ignored attribute cum_table to None
2017-08-10 13:26:19,581 : INFO : loaded dumps/news.model 


Wall time: 5.26 s


In [114]:
%%time
db = lmdb.open('../../data/news data/documents')

url_title = {}

with db.begin() as cur:
    docs = []
    for k, v in cur.cursor():
        docu = Document()
        docu.ParseFromString(v)
        
        url_title[docu.url] = docu.title
        del docu
        if len(url_title) ==100000:
            break

db.close()

Wall time: 8.21 s


In [118]:
def find_doc_from_verb(dictionary, word):
    word2topic = dictionary.word2topic.get_or_create(word)
    
    print('word: {0} \n'.format(word))
    topics_with_proba = [ topic for rank, topic in word2topic.topics.items()][:10]
    
    docs = []
    
    for topic in topics_with_proba:
        topic2doc = dictionary.topic2doc.get_or_create(topic.topic_code)
        docs_with_proba = [ doc for rank, doc in topic2doc.docs.items()][:100]
        for doc in docs_with_proba:
            doc.proba = doc.proba * topic.proba
            docs.append(doc)
    
    docs.sort(key=lambda doc: doc.proba, reverse=True)
    
    largest = docs

    s = set()
    hundred_largest = []
    for n in largest:
        if n.url not in s:
            s.add(n.url)
            hundred_largest.append(n.url)
        if len(hundred_largest) == 101:
            break
    for url in hundred_largest:
        print(url)
#     with db.begin() as txn:
#         for doc in hundred_largest:
#             document = Document()

# #             print(doc)
#             s = txn.get(doc.replace('\n', '').encode('utf-8'))
# #             print(type(s))
#             if s is None:
#                 continue
#             document.ParseFromString(s)
#             if document.title != '':
#                 print(document.title)

In [119]:
find_doc_from_verb(dictionary, '放假')

word: 放假 

http://cul.cn.yahoo.com/newspic/cul/2285/10/

http://chinapic.people.com.cn/redirect.php?goto=findpost=2811475=347332

http://cul.cn.yahoo.com/newspic/cul/2768/26/

http://cul.cn.yahoo.com/newspic/cul/5153/19/

http://chinapic.people.com.cn/redirect.php?goto=findpost=6485910=4529081

http://cpc.people.com.cn/GB/69112/124047/124050/135004/8097574.html

http://ah.people.com.cn/n/2012/0707/c227130-17217608.html

http://cul.cn.yahoo.com/newspic/cul/2458/32/

http://cul.cn.yahoo.com/newspic/cul/2458/33/

http://cul.cn.yahoo.com/newspic/cul/853/33/

http://cul.cn.yahoo.com/newspic/cul/853/32/

http://cul.cn.yahoo.com/newspic/cul/842/6/

http://ah.people.com.cn/BIG5/n/2012/0707/c227130-17217608.html

http://cul.cn.yahoo.com/newspic/cul/842/

http://cul.cn.yahoo.com/newspic/cul/842/1/

http://ah.people.com.cn/n/2012/0615/c228066-17148771.html

http://bj.people.com.cn/n/2012/0708/c82840-17218789.html

http://comic.people.com.cn/n/2012/0716/c144727-18528097.html

http://ah.people.com.

In [80]:
cur = db.begin()

In [19]:
word2topic = dictionary.word2topic.get_or_create('收音机')

In [26]:
word2topic = dictionary.word2topic.get_or_create('放假\n')

In [39]:
from pprint import pprint
pprint(word2topic.topics)

{0: topic_code: 79
proba: 124.14511455524523
, 1: topic_code: 300
proba: 40.71874856649096
, 2: topic_code: 531
proba: 26.84058659150057
, 3: topic_code: 535
proba: 0.001000001534162212
, 4: topic_code: 156
proba: 0.0010000001241843982
, 5: topic_code: 840
proba: 0.0010000000789769654
, 6: topic_code: 740
proba: 0.0010000000510957355
, 7: topic_code: 969
proba: 0.0010000000168033808
, 8: topic_code: 430
proba: 0.0010000000109078536
, 9: topic_code: 939
proba: 0.0010000000086586045
, 10: topic_code: 886
proba: 0.0010000000076688398
, 11: topic_code: 289
proba: 0.0010000000035065523
, 12: topic_code: 802
proba: 0.0010000000033654364
, 13: topic_code: 372
proba: 0.0010000000031550609
, 14: topic_code: 190
proba: 0.0010000000022989458
, 15: topic_code: 475
proba: 0.0010000000020446251
, 16: topic_code: 128
proba: 0.0010000000017828582
, 17: topic_code: 490
proba: 0.001000000001626221
, 18: topic_code: 649
proba: 0.0010000000010682627
, 19: topic_code: 654
proba: 0.001000000001063047
, 20: 

b'sadf'